# Map to region

As explained in https://code.mpimet.mpg.de/boards/2/topics/96

In [23]:
import cdo
import glob
from netCDF4 import Dataset

In [16]:
product = 'bolam'
run = '2018071300'

In [17]:
c = cdo.Cdo()

In [19]:
gribs = sorted(glob.glob('/shared/{}_{}*.grib2'.format(product, run)))
gribs

['/shared/bolam_2018071300_0000000.grib2',
 '/shared/bolam_2018071300_0000100.grib2']

In [22]:
fin = './{}_{}.cf'.format(product, run)
c.cat(input=' '.join(gribs), output=fin,  options = '-r -f nc')
fout = './{}_{}_mapped.cf'.format(product, run)

In [24]:
def dump_nc_attrs(obj, indent=0):
    for k in obj.ncattrs():
        print("%s%s: %s" % (" " * indent, k, obj.getncattr(k)))

In [30]:
orig_ds = Dataset(fin, "r")
temp = orig_ds.variables["t"]
dump_nc_attrs(temp)

standard_name: air_temperature
long_name: Temperature
units: K
param: 0.0.0
grid_mapping: rotated_pole


In [31]:
dump_nc_attrs(orig_ds.variables["rotated_pole"])

grid_mapping_name: rotated_latitude_longitude
grid_north_pole_longitude: -169.5
grid_north_pole_latitude: 46.499996


In [32]:
orig_ds.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  standard_name: time
                  units: hours since 2018-7-13 00:00:00
                  calendar: proleptic_gregorian
                  axis: T
              unlimited dimensions: time
              current shape = (2,)
              filling off), ('rlon', <class 'netCDF4._netCDF4.Variable'>
              float64 rlon(rlon)
                  standard_name: grid_longitude
                  long_name: longitude in rotated pole grid
                  units: degrees
                  axis: X
              unlimited dimensions: 
              current shape = (578,)
              filling off), ('rlat', <class 'netCDF4._netCDF4.Variable'>
              float64 rlat(rlat)
                  standard_name: grid_latitude
                  long_name: latitude in rotated pole grid
                  units: degrees
                  axis: Y
              unlimited dimensions: 
          

In [33]:
orig_ds.close()

In [27]:
xsize = 256
ysize = 256
xfirst = 6
xlast  = 19
xinc = (xlast - xfirst)/xsize
yfirst = 36
ylast  = 47
yinc = (ylast - yfirst)/ysize

gfname = 'mygrid.txt'
with open(gfname, 'w') as o:
    o.write("""
    gridtype = lonlat
    xsize = {}
    ysize = {}
    xfirst = {}
    xinc = {}
    yfirst = {}
    yinc = {}
    """.format(xsize, ysize, xfirst, xinc, yfirst, yinc))
c.remapbil(gfname, input=fin, output=fout)

'./bolam_2018071300_mapped.cf'

In [29]:
mapped_ds = Dataset(fout, "r")
temp = mapped_ds.variables["t"]
dump_nc_attrs(temp)

standard_name: air_temperature
long_name: Temperature
units: K
param: 0.0.0


In [34]:
mapped_ds.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  standard_name: time
                  units: hours since 2018-7-13 00:00:00
                  calendar: proleptic_gregorian
                  axis: T
              unlimited dimensions: time
              current shape = (2,)
              filling off), ('lon', <class 'netCDF4._netCDF4.Variable'>
              float64 lon(lon)
                  standard_name: longitude
                  long_name: longitude
                  units: degrees_east
                  axis: X
              unlimited dimensions: 
              current shape = (256,)
              filling off), ('lat', <class 'netCDF4._netCDF4.Variable'>
              float64 lat(lat)
                  standard_name: latitude
                  long_name: latitude
                  units: degrees_north
                  axis: Y
              unlimited dimensions: 
              current shape = (256,)
              fillin